In [1]:
# Biblotecas de manipulação de dados
import numpy as np
import pandas as pd
# Bibliotecas de visualização
import seaborn as sns
import matplotlib.pyplot as plt
# Testes estatísticos
from scipy.stats import ttest_ind


from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn import datasets, linear_model
from sklearn.metrics import classification_report 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Bibliotecas Locais
from carga import *
from tratamentos import *
from categorizacao import *

pfEnade = doCargaTodos('microdados_enade_2019.txt')
# TRATAMENTOS 
pfEnade = doIdade(pfEnade)
pfEnade = doNota(pfEnade)
pfEnade = doVazios(pfEnade)
pfEnade = doPublicoPrivada(pfEnade)
pfEnade = doHumanasExatas(pfEnade)
pfEnade = doRenda(pfEnade)
pfEnade = doMigracao(pfEnade)
pfEnade = doRenda(pfEnade)
pfEnade = doEscolaridade(pfEnade)
pfEnade = doCategoricos(pfEnade)
pfEnade = doRaiz(pfEnade)


AttributeError: Can only use .cat accessor with a 'category' dtype

In [ ]:
pfEnade.dataframeName = 'dados_enade.csv'
nRow, nCol = pfEnade.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
# CONTAR CLASSES
count_class_0, count_class_1 = pfEnade['Publica'].value_counts()# pfEnade.target.value_counts()
# Divide by class
df_class_0 = pfEnade[pfEnade['Publica'] == 1]
df_class_1 = pfEnade[pfEnade['Publica'] == 0]
df_class_0_under = df_class_0.sample(count_class_1)
dfAjustado = pd.concat([df_class_0_under, df_class_1], axis=0)

enade_features = ['BomAluno','CO_REGIAO_CURSO', 'Jovem', 'Migrado', 'Solo', 'Exatas', 'QE_I23CAT', 'BaixaRenda', 'Sustento', 'Sexo']
enade_featuresRaiz = ['BomAlunoRAIZRAIZ','CO_REGIAO_CURSORAIZ', 'JovemRAIZ', 'MigradoRAIZ', 'SoloRAIZ', 'ExatasRAIZ', 'QE_I23CATRAIZ', 'BaixaRendaRAIZ', 'SustentoRAIZ', 'SexoRAIZ']

#age_sex = titanic[["Age", "Sex"]]
#dfAjustado = dfAjustado[]

dfAjustado1 = pd.DataFrame(dfAjustado, columns = enade_features)
dfAjustado1.dataframeName = 'Categorized'

dfAjustadoRaiz = pd.DataFrame(dfAjustado, columns = enade_featuresRaiz)
dfAjustadoRaiz.dataframeName = 'ROOT'

In [ ]:
plotCorrelationMatrix(dfAjustado1, 8)

In [ ]:
plotCorrelationMatrix(dfAjustadoRaiz, 8)

In [ ]:
plotCorrelationMatrix(pfEnade, 15)

In [ ]:
plotScatterMatrix(pfEnade, 20, 10)